# PROYECTO FINAL
## Modelos Computacionales II
### Estela Gil Villegas Guevara

### Parte I - Análisis de Dataset

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from skimage import img_as_ubyte
from scipy.stats import entropy, skew, kurtosis
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

Para este proyecto, opté por usar las métricas de las imágenes (kurtosis, homogeneidad, disimilitud, correlación, contraste, skewness, entropía, energía), para sacar un PCA y así:

* Determinar qué características son más relevantes y proporcionan mayor información para clasificar a las imágenes
* Disminuir la carga de mi computadora ;) y evitar más problemas con tensor flow

*Nota: regresaron los problemas de instalación de paquetes, pero evaluando el código en otra computadora, verifiqué los resultados*

In [ ]:
data = []

for set_name in ['Training', 'Testing']: # iterar sobre cada carpeta de train y test
    set_path = os.path.join(base_path, set_name)
    
    for class_name in ['glioma', 'meningioma', 'notumor', 'pituitary']:
        class_path = os.path.join(set_path, class_name)
        
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            if img is None:
                continue

            # extraer estadísticos y medidas de las imágenes
            mean_val = np.mean(img)
            std_val = np.std(img)
            entropy_val = entropy(img.ravel())
            skewness_val = skew(img.ravel())
            kurtosis_val = kurtosis(img.ravel())
            contrast_val = np.max(img) - np.min(img)
            glcm = graycomatrix(img_as_ubyte(img), distances=[1], angles=[0], symmetric=True, normed=True)
            energy_val = graycoprops(glcm, 'energy')[0, 0]
            asm_val = energy_val ** 2
            homogeneity_val = graycoprops(glcm, 'homogeneity')[0, 0]
            correlation_val = graycoprops(glcm, 'correlation')[0, 0]
            dissimilarity_val = np.mean(np.abs(img - np.mean(img)))

            data.append([filename, set_name, class_name, mean_val, std_val, entropy_val, skewness_val, kurtosis_val,
                         contrast_val, energy_val, asm_val, homogeneity_val, dissimilarity_val, correlation_val]) # guardar todo en nuestra lista

In [ ]:
df = pd.DataFrame(data, columns=['NombreImagen', 'Set', 'Clase', 'Mean', 'SD', 'Entropia', 'Skewness', 'Kurtosis',
                                 'Contrast', 'Energy', 'ASM', 'Homogeneidad', 'Dissimilitud', 'Correlacion']) # data frame con los resultados de las medidas

In [ ]:
X = df.iloc[:, 3:]  # obtener las columnas (excepto nombre imagen, set y clase, no nos interesan
y = df['Clase'] # obtener etiqueta de clasificación

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # dividir en train y test

In [ ]:
pca = PCA(n_components=5) # usar PCA para obtener las características más relevantes
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
df_train_pca = pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(5)]) # dataframe de datos transformados (train)
df_test_pca = pd.DataFrame(X_test_pca, columns=[f'PC{i+1}' for i in range(5)]) # dataframe de datos transformados (test)

In [ ]:
# añadir etiquetas nuevamente
df_train_pca['Clase'] = y_train.values
df_test_pca['Clase'] = y_test.values

In [1]:
# guardar los nuevos datasets (archivos csv)
df_train_pca.to_csv('dataset_train_pca.csv', index=False)
df_test_pca.to_csv('dataset_test_pca.csv', index=False)

print("el dataset de entrenamiento y prueba con PCA se guardaron.")

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.10 from "C:\Users\estel\OneDrive\Escritorio\NEUROCIENCIAS\sexto_sem\modelos_python\Neurociencias-2025-2\dl_env\Scripts\python.exe"
  * The NumPy version is: "1.24.4"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: No se puede encontrar el módulo especificado.


ImportError: numpy._core.multiarray failed to import

### Parte II - Identificación de tipo de problema
En este caso, ya que el dataset contiene datos de distintos tipos de tumor vs no tumor, el tipo de problema es de **clasificación**: entrenar el modelo para que pueda determinar si el dato corresponde a un tipo de tumor o no. 

### Parte III - Propuesta de Modelos

In [6]:
df_train = pd.read_csv('dataset_train_pca.csv')
df_test = pd.read_csv('dataset_test_pca.csv')

In [7]:
X_train = df_train.iloc[:, :-1] # separa características (train)
y_train = df_train.iloc[:, -1] # separa etiquetas (train)
X_test = df_test.iloc[:, :-1] # separa etiquetas (test)
y_test = df_test.iloc[:, -1]# separa etiquetas (test)

In [8]:
# convertir etiquetas a valores numéricos
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

#### Modelo 1 - Redes Neuronales Artificiales

In [9]:
model = keras.Sequential([
    keras.layers.Dense(128),
    keras.layers.LeakyReLU(alpha=0.01),  # leaky ReLU en vez de ReLU
    keras.layers.BatchNormalization(), 
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(64),
    keras.layers.LeakyReLU(alpha=0.01),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(32),
    keras.layers.LeakyReLU(alpha=0.01),
    
    keras.layers.Dense(len(np.unique(y_train)), activation='softmax')  
])

In [10]:
optimizer = keras.optimizers.Adam(learning_rate=0.0003)  # bajar la tasa de aprendizaje
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# entrenamiento con + épocas
model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/150
176/176 [==============================] - 4s 12ms/step - loss: 1.1770 - accuracy: 0.5203 - val_loss: 0.8687 - val_accuracy: 0.6591
Epoch 2/150
176/176 [==============================] - 1s 5ms/step - loss: 0.9413 - accuracy: 0.6331 - val_loss: 0.7888 - val_accuracy: 0.6833
Epoch 3/150
176/176 [==============================] - 1s 5ms/step - loss: 0.8686 - accuracy: 0.6600 - val_loss: 0.7449 - val_accuracy: 0.7032
Epoch 4/150
176/176 [==============================] - 1s 7ms/step - loss: 0.8357 - accuracy: 0.6727 - val_loss: 0.7137 - val_accuracy: 0.7117
Epoch 5/150
176/176 [==============================] - 1s 7ms/step - loss: 0.8143 - accuracy: 0.6785 - val_loss: 0.6981 - val_accuracy: 0.7196
Epoch 6/150
176/176 [==============================] - 1s 6ms/step - loss: 0.7951 - accuracy: 0.6803 - val_loss: 0.6765 - val_accuracy: 0.7217
Epoch 7/150
176/176 [==============================] - 1s 7ms/step - loss: 0.7929 - accuracy: 0.6812 - val_loss: 0.6673 - val_accuracy: 0.728

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test, y_pred) # evaluar rendimiento
print(f'Accuracy mejorada en prueba: {accuracy:.2f}')

#### Modelo 2 - Bosques Aleatorios

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42) # definir modelo de bosque aleatorio (clasificador)
rf_model.fit(X_train, y_train) # entrenamiento del modelo

y_pred = rf_model.predict(X_test) # predicciones

In [ ]:
accuracy = accuracy_score(y_test, y_pred) # evaluar rendimiento
print(f'Accuracy del modelo Bosques Aleatorios en prueba: {accuracy:.2f}')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# predicciones RNA
y_pred_rna = model.predict(X_test)
y_pred_rna = np.argmax(y_pred_rna, axis=1)  

# predicciones bosque aleatorio
y_pred_rf = rf_model.predict(X_test)

In [ ]:
# sacar métricas
metrics = {}

for model_name, y_pred in zip(["RNA", "Random Forest"], [y_pred_rna, y_pred_rf]):
    metrics[model_name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "F1-score": f1_score(y_test, y_pred, average="weighted"),
        "Recall": recall_score(y_test, y_pred, average="weighted"),
        "Precisión": precision_score(y_test, y_pred, average="weighted")
    }

# 🔹 Mostrar métricas
for model_name, values in metrics.items():
    print(f"\n🔍 Métricas de {model_name}:")
    for metric, value in values.items():
        print(f"{metric}: {value:.2f}")

In [ ]:
# matriz de confusión - RNA
plt.figure(figsize=(6,5))
sns.heatmap(confusion_matrix(y_test, y_pred_rna), annot=True, fmt='d', cmap='viridis')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - RNA')
plt.show()

In [ ]:
#  matriz de confusión - bosque aleatorio
plt.figure(figsize=(6,5))
sns.heatmap(confusion_matrix(y_test, y_pred_rf), annot=True, fmt='d', cmap='coolwarm')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión - Random Forest')
plt.show()

En este caso, analicé el desempeño de las Redes Neuronales Artificiales (RNA) y los Bosques Aleatorios (BA).

El objetivo fue evaluar cuál de estos métodos ofrece mejor rendimiento en un conjunto de datos con características extraídas de imágenes, (tales como contraste, kurtosis, entropía y energía). Para ello, comparé ambos modelos bajo métricas de precisión, capacidad de generalización y robustez frente a datos no vistos.

Por un lado, opté por las RNA, ya que pueden detectar interacciones sutiles entre características como textura y contraste, son eficientes en casos donde las variables presentan alta dimensionalidad o relaciones complejas (como sucede con algunas de las variables), y además se adaptan bien a la variabilidad significativa en los datos.
Por otro lado, los bosques aleatorios permiten evaluar la importancia de cada característica, ayudando a entender cuáles son más relevantes para la clasificación, lo cual era de mi interés en este contexto.  

Hubiera esperado que las redes neuronales clasificaran con mayor precisión, pero los bosques aleatorios fueron mejores, lo cual se podría explicar si las características extraídas realmente lo suficientemente discriminatorias entre sí. Igualmente, si solamente hay pequeñas variaciones, los bosques aleatorios tendrían mayor capacidad de realizar predicciones más estables. 